In [1]:

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter



# Read the clean_flavors_of_cacao.csv

In [2]:
# Load the data
file_path = Path('clean_flavors_of_cacao.csv')
df = pd.read_csv(file_path)
df.head()


,Company,Bean_Origin_or_Bar_Name,REF,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin
0,A. Morin,Agua Grande,1876,2016,63.0,France,3.75,missing,Sao Tome & Principe
1,A. Morin,Kpime,1676,2015,70.0,France,2.75,missing,Togo
2,A. Morin,Atsane,1676,2015,70.0,France,3.00,missing,Togo
3,A. Morin,Akata,1680,2015,70.0,France,3.50,missing,Togo
4,A. Morin,Quilla,1704,2015,70.0,France,3.50,missing,Peru


In [3]:
df= df.drop(columns=["REF","Bean_Origin_or_Bar_Name"], axis=1)
df.head()

,Company,Review_Date,Cocoa_Percent,Company_Location,Rating,Bean_Type,Broad_Bean_Origin
0,A. Morin,2016,63.0,France,3.75,missing,Sao Tome & Principe
1,A. Morin,2015,70.0,France,2.75,missing,Togo
2,A. Morin,2015,70.0,France,3.00,missing,Togo
3,A. Morin,2015,70.0,France,3.50,missing,Togo
4,A. Morin,2015,70.0,France,3.50,missing,Peru


In [4]:
df['Rating'].describe()


count    1787.000000
mean        3.186766
std         0.477579
min         1.000000
25%         3.000000
50%         3.250000
75%         3.500000
max         5.000000
Name: Rating, dtype: float64

In [5]:
def bin_ratings(rating):
    if rating == 5.75: return 1
    if rating == 5.50: return 1
    if rating == 5.25: return 1
    if rating == 5.00: return 1
    
    if rating == 4.75: return 1
    if rating == 4.50: return 1
    if rating == 4.25: return 1
    if rating == 4.00: return 1
    
    if rating == 3.75: return 1
    if rating == 3.50: return 1
    if rating == 3.25: return 0
    if rating == 3.00: return 0

    if rating == 2.75: return 0
    if rating == 2.50: return 0
    if rating == 2.25: return 0
    if rating == 2.00: return 0
    
    if rating == 1.75: return 0
    if rating == 1.50: return 0
    if rating == 1.25: return 0
    if rating == 1.00: return 0
    
    #print( f"error: rating={rating} type={type(rating)}" )
    return "2"

In [6]:
df['Rating'] = df['Rating'].apply(bin_ratings)

# Split the Data into Training and Testing


In [7]:
y = df["Rating"]
X = df.drop(columns="Rating")
X = pd.get_dummies(X)

In [8]:
X.describe()

,Review_Date,Cocoa_Percent,Company_A. Morin,Company_AMMA,Company_Acalli,Company_Adi,Company_Aequare (Gianduja),Company_Ah Cacao,Company_Akesson's (Pralus),Company_Alain Ducasse,...,Broad_Bean_Origin_Sri Lanka,Broad_Bean_Origin_Suriname,Broad_Bean_Origin_Tanzania,Broad_Bean_Origin_Togo,Broad_Bean_Origin_Trinidad Tobago,Broad_Bean_Origin_Uganda,Broad_Bean_Origin_Unknown,Broad_Bean_Origin_Vanuatu,Broad_Bean_Origin_Venezuela,Broad_Bean_Origin_Vietnam
count,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,...,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000,1787.000000
mean,2012.324566,71.701735,0.012871,0.002798,0.001119,0.002238,0.001119,0.000560,0.001679,0.002798,...,0.001119,0.000560,0.019026,0.001679,0.021824,0.003917,0.040851,0.003917,0.125350,0.021265
std,2.930680,6.331193,0.112748,0.052837,0.033445,0.047272,0.033445,0.023656,0.040950,0.052837,...,0.033445,0.023656,0.136656,0.040950,0.146150,0.062482,0.197999,0.062482,0.331208,0.144306
min,2006.000000,42.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2010.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2013.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2015.000000,75.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2017.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# Check the balance of our target values
y.value_counts()

0    1087
1     700
Name: Rating, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,test_size= 0.4,train_size=0.6 , random_state=1, stratify=y)

In [11]:

Counter(y_train)

Counter({0: 652, 1: 420})

In [12]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Oversampling

In [13]:
#Naive Random Oversampling

# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
# Instantiate the model
ros = RandomOverSampler(random_state=1)
# Resample the targets
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)



Counter({0: 652, 1: 652})

In [14]:
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
# Calculate predictions
y_pred = model.predict(X_test_scaled)


In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6575738916256157

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,294,141
Actual low_risk,101,179


In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.68      0.64      0.71      0.66      0.43       435
          1       0.56      0.64      0.68      0.60      0.66      0.43       280

avg / total       0.67      0.66      0.65      0.66      0.66      0.43       715



# SMOTE Oversampling


In [18]:


# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,sampling_strategy='auto').fit_resample(X_train_scaled, y_train)

Counter(y_resampled)



Counter({0: 652, 1: 652})

In [19]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.6574507389162562

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[297, 138],
       [103, 177]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.68      0.63      0.71      0.66      0.43       435
          1       0.56      0.63      0.68      0.59      0.66      0.43       280

avg / total       0.67      0.66      0.65      0.67      0.66      0.43       715



# Undersampling

In [23]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids
# Instantiate
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 420, 1: 420})

In [24]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.6626642036124795

In [26]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,286,149
Actual low_risk,93,187


In [27]:


# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))



                   pre       rec       spe        f1       geo       iba       sup

          0       0.75      0.66      0.67      0.70      0.66      0.44       435
          1       0.56      0.67      0.66      0.61      0.66      0.44       280

avg / total       0.68      0.66      0.66      0.67      0.66      0.44       715



# Combination (Over and Under) Sampling 

In [28]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_scaled, y_train)
Counter(y_resampled)

Counter({0: 109, 1: 327})

In [29]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [30]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.5300697865353038

In [31]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,65,370
Actual low_risk,25,255


In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.72      0.15      0.91      0.25      0.37      0.13       435
          1       0.41      0.91      0.15      0.56      0.37      0.15       280

avg / total       0.60      0.45      0.61      0.37      0.37      0.13       715

